In [127]:
from dash import Dash, html, dcc, Input, Output, State
from dash.dash_table import DataTable
from sqlalchemy import create_engine, text
import pymysql
import pandas as pd

In [42]:
pro = 'mysql+pymysql://'
user = 'bigdata'
password = 'Bigdata123!!'
host = '192.168.56.101'
port = '3306'
db = 'SampleDB'
engine = create_engine(f'{pro}{user}:{password}@{host}:{port}/{db}')
conn = engine.connect()

In [97]:
## 데이터 베이스 목록 가져오기
db_list = pd.read_sql("select distinct table_schema from information_schema.tables", conn).loc[:,'table_schema']
db_list = db_list.values

In [128]:
app = Dash(__name__)
app.layout = html.Div(
    [
        dcc.Dropdown(id='db-list-dropdown',
                    options = [{'label':i, 'value': i} for i in db_list]),
        html.Br(),
        html.Div(id='table-dropdown'),
        html.Div(id='column-chekcklist'),
        html.Div(id='condition-column'),
        html.Div(id='condition-select'),
        dcc.Input(id='input-value',type='text'),
        html.Button('조회',id='submit',n_clicks=0),
        html.Hr(),
        html.Div(id='table-contents')
    ]
)

In [130]:
@app.callback(
    Output('table-dropdown','children'),
    Input('db-list-dropdown','value')
)
def update_table_dropdown_list(dbname):
    table_list = pd.read_sql(f"select table_name from information_schema.tables where table_schema = '{dbname}'", conn)
    table_list = table_list.loc[:,'table_name'].values
    table_list = [{'label':i,'value':i} for i in table_list]
    return html.Div(
        [
            dcc.Dropdown(id = 'table-list-dropdown',
                        options = table_list)
        ]
    )

@app.callback(
    Output('column-chekcklist','children'),
    Input('db-list-dropdown','value'),
    Input('table-list-dropdown','value')
)
def update_column_list(dbname,tablename):
    column_list = pd.read_sql(f"select column_name from information_schema.columns where table_schema = '{dbname}' and table_name = '{tablename}'", conn)
    column_list = column_list.loc[:,'column_name']
    column_list = column_list.values
    return html.Div(
        [
            dcc.Checklist(
                id = 'column-list',
                options = column_list,
                value = column_list[[0]]
            )
        ]
    )

@app.callback(
    Output('condition-column','children'),
    Input('db-list-dropdown','value'),
    Input('table-list-dropdown','value')
)
def update_condition_column_list(dbname, tablename):
    condition_column_list = pd.read_sql(f"select column_name from information_schema.columns where table_schema = '{dbname}' and table_name = '{tablename}'", conn)
    condition_column_list = condition_column_list.loc[:,'column_name'].values
    return html.Div(
        [
            dcc.RadioItems(
                id = 'condition-column-radio',
                options = condition_column_list
            )
        ]
    )

@app.callback(
    Output('condition-select','children'),
    Input('condition-column-radio','value')
)
def update_select_condition(condition_column):
    
    condition_list = [
        {'label':'=','value':'='},
        {'label':'!=','value':'!='},
        {'label':'선택안함','value':'no_selection'}
    ]
    return html.Div(
        [
            dcc.RadioItems(
            id = 'condition-radio',
            options = condition_list)
        ]
    )


@app.callback(
    Output('table-contents','children'),
    Input('submit','n_clicks'),
    State('db-list-dropdown','value'),
    State('table-list-dropdown','value'),
    State('column-list','value'),
    State('condition-column-radio','value'),
    State('condition-radio', 'value'),
    State('input-value','value')
)
def update_table_contents(n,dbname,tablename,columnname,condition_col,condition_radio,input_val):
    columnname = ','.join(columnname)
    if len(columnname) == 0:
        columnname = '*'
    if input_val == 'no_selection':
        stmt = f"select {columnname} from {dbname}.{tablename}"
    else:
        stmt = f"select {columnname} from {dbname}.{tablename} where {condition_col} {condition_radio} '{input_val}'"
    table_contents = pd.read_sql(stmt, conn)
    return html.Div(
        [
            DataTable(
                table_contents.to_dict('records'),
                      [{'name': i, 'id': i} for i in table_contents.columns]
            )
        ]
    )

In [131]:
app.run_server(host='0.0.0.0', port=7777)

Dash is running on http://192.168.56.101:5555/

Dash is running on http://192.168.56.101:5555/

Dash is running on http://192.168.56.101:5555/

Dash is running on http://192.168.56.101:5555/

Dash is running on http://192.168.56.101:5555/

Dash is running on http://192.168.56.101:5555/

Dash is running on http://192.168.56.101:5555/

Dash is running on http://192.168.56.101:5555/

Dash is running on http://192.168.56.101:5555/

Dash is running on http://192.168.56.101:5555/

Dash is running on http://192.168.56.101:5555/

Dash is running on http://192.168.56.101:5555/

Dash is running on http://192.168.56.101:5555/

Dash is running on http://192.168.56.101:5555/

Dash is running on http://192.168.56.101:5555/

Dash is running on http://192.168.56.101:5555/

Dash is running on http://192.168.56.101:5555/

Dash is running on http://192.168.56.101:5555/

Dash is running on http://192.168.56.101:5555/

Dash is running on http://192.168.56.101:5555/

Dash is running on http://192.168.56.101

 * Running on http://192.168.56.101:5555/ (Press CTRL+C to quit)
192.168.56.1 - - [22/Feb/2022 13:45:25] "GET / HTTP/1.1" 200 -
192.168.56.1 - - [22/Feb/2022 13:45:26] "GET /_dash-layout HTTP/1.1" 200 -
192.168.56.1 - - [22/Feb/2022 13:45:26] "GET /_dash-dependencies HTTP/1.1" 200 -
192.168.56.1 - - [22/Feb/2022 13:45:26] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 200 -
192.168.56.1 - - [22/Feb/2022 13:45:26] "POST /_dash-update-component HTTP/1.1" 200 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/home/ejchoi/anaconda3/lib/python3.9/site-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/home/ejchoi/anaconda3/lib/python3.9/site-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/home/ejchoi/anaconda3/lib/python3.9/site-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/home/ejchoi/anaconda3/lib/python3.9/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/home/ejchoi/anaconda3/lib/python3.9/site-packages/flask/app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "/home/ejchoi/anaconda3/lib/python3.9/site-packages/flask/app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "/home/ejchoi/anaconda3/lib/python3.9/site-packages/

192.168.56.1 - - [22/Feb/2022 13:45:26] "POST /_dash-update-component HTTP/1.1" 500 -
192.168.56.1 - - [22/Feb/2022 13:45:26] "POST /_dash-update-component HTTP/1.1" 200 -
192.168.56.1 - - [22/Feb/2022 13:45:26] "POST /_dash-update-component HTTP/1.1" 200 -
192.168.56.1 - - [22/Feb/2022 13:45:31] "POST /_dash-update-component HTTP/1.1" 200 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/home/ejchoi/anaconda3/lib/python3.9/site-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/home/ejchoi/anaconda3/lib/python3.9/site-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/home/ejchoi/anaconda3/lib/python3.9/site-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/home/ejchoi/anaconda3/lib/python3.9/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/home/ejchoi/anaconda3/lib/python3.9/site-packages/flask/app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "/home/ejchoi/anaconda3/lib/python3.9/site-packages/flask/app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "/home/ejchoi/anaconda3/lib/python3.9/site-packages/

...

192.168.56.1 - - [22/Feb/2022 13:48:37] "POST /_dash-update-component HTTP/1.1" 500 -
192.168.56.1 - - [22/Feb/2022 13:48:44] "POST /_dash-update-component HTTP/1.1" 200 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/home/ejchoi/anaconda3/lib/python3.9/site-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/home/ejchoi/anaconda3/lib/python3.9/site-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/home/ejchoi/anaconda3/lib/python3.9/site-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/home/ejchoi/anaconda3/lib/python3.9/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/home/ejchoi/anaconda3/lib/python3.9/site-packages/flask/app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "/home/ejchoi/anaconda3/lib/python3.9/site-packages/flask/app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "/home/ejchoi/anaconda3/lib/python3.9/site-packages/

192.168.56.1 - - [22/Feb/2022 13:48:44] "POST /_dash-update-component HTTP/1.1" 500 -
192.168.56.1 - - [22/Feb/2022 13:48:44] "POST /_dash-update-component HTTP/1.1" 200 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/home/ejchoi/anaconda3/lib/python3.9/site-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/home/ejchoi/anaconda3/lib/python3.9/site-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/home/ejchoi/anaconda3/lib/python3.9/site-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/home/ejchoi/anaconda3/lib/python3.9/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/home/ejchoi/anaconda3/lib/python3.9/site-packages/flask/app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "/home/ejchoi/anaconda3/lib/python3.9/site-packages/flask/app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "/home/ejchoi/anaconda3/lib/python3.9/site-packages/

192.168.56.1 - - [22/Feb/2022 13:48:44] "POST /_dash-update-component HTTP/1.1" 500 -
192.168.56.1 - - [22/Feb/2022 13:48:44] "POST /_dash-update-component HTTP/1.1" 200 -
192.168.56.1 - - [22/Feb/2022 13:48:44] "POST /_dash-update-component HTTP/1.1" 200 -
192.168.56.1 - - [22/Feb/2022 13:48:49] "POST /_dash-update-component HTTP/1.1" 200 -
192.168.56.1 - - [22/Feb/2022 13:48:49] "POST /_dash-update-component HTTP/1.1" 200 -
192.168.56.1 - - [22/Feb/2022 13:48:49] "POST /_dash-update-component HTTP/1.1" 200 -
192.168.56.1 - - [22/Feb/2022 13:49:02] "POST /_dash-update-component HTTP/1.1" 200 -
192.168.56.1 - - [22/Feb/2022 13:49:15] "POST /_dash-update-component HTTP/1.1" 200 -


In [10]:
# df = pd.read_sql("select * from world", conn)

# app = Dash(__name__)
# app.layout = html.Div(
#     [
#         DataTable(df.to_dict('record'),
#                   [{'name': i, 'id': i} for i in df.columns],
#                     editable=False,
#                     filter_action="native",
#                     sort_action="native",
#                     sort_mode="multi",
#                     row_selectable="multi",
#                     row_deletable=False,
#                     selected_rows=[],
#                     page_action="native",
#                     page_current= 0,
#                     page_size= 6,
#                     # page_action='none',
#                     # style_cell={
#                     # 'whiteSpace': 'normal'
#                     # },
#                     # fixed_rows={ 'headers': True, 'data': 0 },
#                     # virtualization=False,
#                     style_cell_conditional=[
#                         {'if': {'column_id': 'countriesAndTerritories'},
#                          'width': '40%', 'textAlign': 'left'},
#                         {'if': {'column_id': 'deaths'},
#                          'width': '30%', 'textAlign': 'left'},
#                         {'if': {'column_id': 'cases'},
#                          'width': '30%', 'textAlign': 'left'},
#                         ]
#                  )
#     ]
# )

# app.run_server(host='192.168.56.101', port=7777)